# Generate embeddings of the cleaned dataset

Import necessary libraries and load the data
> Remember to have an OpenAI key, you can get one at https://platform.openai.com/account/api-keys

In [3]:
# pyright: reportGeneralTypeIssues=false

import pandas as pd 
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import json
from openai import OpenAI
from dotenv import load_dotenv
import os
from tqdm import tqdm
import math
import time

load_dotenv()

OPENAI_KEY = os.getenv("OPENAI_KEY")

Load the datasets of the **green skills** provided by `ESCO`

In [4]:
df_green_skills = pd.read_csv("../data/taxonomies/esco_green_skills_cleaned.csv")
df_green_skills.head()

,green_skill,alt_label,description
0,train staff to reduce food waste,teach students food waste reduction practices,Establish new trainings and staff development ...
1,train staff to reduce food waste,inform staff on food waste reduction practices,Establish new trainings and staff development ...
2,train staff to reduce food waste,educate workers on food recycling methods,Establish new trainings and staff development ...
3,train staff to reduce food waste,educate staff on food waste reduction,Establish new trainings and staff development ...
4,develop energy saving concepts,create concepts for energy saving,Use current research results and collaborate w...


Concatenate the `Main Name`, `Alternative Name` and `Description` columns to create a single text representation for each skill.
This will be used to generate richer embeddings.

In [33]:
for _, row in df_green_skills.head(10).iterrows():
    label = row["green_skill"] + ". " + row["alt_label"] + ". " + row["description"]
    print(label)
    # train staff to reduce food waste. teach students food waste reduction practices. Establish new trainings and staff development provisions to support staff knowledge in food waste prevention and food recycling practices. Ensure that staff understands methods of and tools for food recycling, e.g., separating waste.

train staff to reduce food waste. teach students food waste reduction practices. Establish new trainings and staff development provisions to support staff knowledge in food waste prevention and food recycling practices. Ensure that staff understands methods of and tools for food recycling, e.g., separating waste.
train staff to reduce food waste. inform staff on food waste reduction practices. Establish new trainings and staff development provisions to support staff knowledge in food waste prevention and food recycling practices. Ensure that staff understands methods of and tools for food recycling, e.g., separating waste.
train staff to reduce food waste. educate workers on food recycling methods. Establish new trainings and staff development provisions to support staff knowledge in food waste prevention and food recycling practices. Ensure that staff understands methods of and tools for food recycling, e.g., separating waste.
train staff to reduce food waste. educate staff on food wa

## Embeddings of the green skills
Here we are generating the embeddings of the green skill dataset using the `text-embedding-3-large` model from **OpenAI**. The embeddings will be stored in a **FAISS** index for efficient similarity search using **cosine similarity** as metric.

In [ ]:
client = OpenAI(api_key = OPENAI_KEY)

embeddings = []

for row in tqdm(df_green_skills.iterrows(), total=df_green_skills.shape[0], desc="Generating embeddings", unit="embedding"):
    text = row[1]["green_skill"] + ". " + row[1]["alt_label"] + ". " + row[1]["description"]
    
    try:
        response = client.embeddings.create(
            input = text,
            model = "text-embedding-3-large"
        )
        embeddings.append(response.data[0].embedding)
    except Exception as e:
        embeddings.append([0.0]*3072)  # Append a zero vector in case of error
        send_warning(f"Error generating embedding for row {row[0]}: {e}")
        time.sleep(5)  # Wait before retrying to avoid hitting rate limits

embeddings[:2]

Generating embeddings:   0%|          | 0/2539 [00:00<?, ?embedding/s]

Generating embeddings: 100%|██████████| 2539/2539 [12:54<00:00,  3.28embedding/s]  


[[-0.04066907614469528,
  0.018801070749759674,
  -0.015321778133511543,
  0.008852869272232056,
  -0.020282993093132973,
  -0.023414356634020805,
  0.049070924520492554,
  -0.013556358404457569,
  -0.016752153635025024,
  0.017074311152100563,
  -0.0025949731934815645,
  -0.027164261788129807,
  -0.014471284113824368,
  0.017357809469103813,
  0.016765039414167404,
  -0.040308259427547455,
  0.012705864384770393,
  0.01338883675634861,
  0.04711221158504486,
  -0.028272481635212898,
  0.045565858483314514,
  -0.05623569339513779,
  -0.017100082710385323,
  -0.009336104616522789,
  -0.008376076817512512,
  0.021700482815504074,
  -0.016790812835097313,
  0.0038626601453870535,
  -0.046416353434324265,
  0.022215932607650757,
  0.027293125167489052,
  0.039122723042964935,
  -0.025695227086544037,
  -0.06860651075839996,
  -0.033865123987197876,
  0.010811582207679749,
  0.042086564004421234,
  -0.00916858296841383,
  -0.01597897708415985,
  0.018504686653614044,
  -0.0388907715678215,


Creating the `FAISS` index and adding the embeddings to it

In [28]:
index = faiss.IndexFlatIP(embeddings[0].__len__())
embeddings = np.array(embeddings).astype('float32')
index.add(embeddings)  

Here we are creating a **dictionary** of tuples in the form `(i, (skill, text))`, where: 
* `i` is the unique key for that entry, useful because **FAISS** only stores the *vector embedding*, not the actual text, so we need to save the identifier to map the vector to the entry.
* `(skill, text)`: a tuple with two elements, `skill` is the official name of the green skill given by the **esco taxonomy**, `text` is the alternative label detected in the semantic search, the same **green skill** can be retrieved as a result of similar job descriptions.

In [18]:
id_to_skill = {i: (skill, text) for i, (skill, text) in enumerate(zip(df_green_skills["green_skill"], df_green_skills["alt_label"]))}
id_to_skill[0]

('train staff to reduce food waste',
 'teach students food waste reduction practices')

Testing the index

In [ ]:
query = "Ayudar a la reforestación"  # Test query

query_embedding = client.embeddings.create(
    input=query,
    model="text-embedding-3-large"
).data[0].embedding

query_vec = np.array([query_embedding], dtype=np.float32)

D, I = index.search(query_vec, k=2)

for i, score in zip(I[0], D[0]):
    print(f"Score: {score:.4f}\nSkill: {id_to_skill[i][0]}\nText: {id_to_skill[i][1]}\n")

Score: 0.4316
Skill: plant trees
Text: transplant trees

Score: 0.4180
Skill: care for the wildlife
Text: flora and fauna protecting



Saving the **index** and the **mapping**

In [30]:
faiss.write_index(index, "../data/embeddings/esco_green_skills_text-embedding-3-large.index")
with open("../data/mapping/id_to_skill.json", "w") as f:
    json.dump(id_to_skill, f, indent=4)


Ensuring the **index** and **mapping** were saved correctly. Given that there is a correspondence between the index and the mapping, they should have the same length.

In [ ]:
print(len(id_to_skill)) # Length of the mapping
print(df_green_skills.shape) # Shape of the dataframe
print(index.ntotal) # Number of vectors in the index

# They should be the same
print(len(id_to_skill) == df_green_skills.shape[0] == index.ntotal)

2539
(2539, 3)
2539
True


## Embeddings of jobs from january 2025 to april 2025
Here we are generating the embeddings of the job descriptions from january 2025 to april 2025, this will be used later for predicting green skills in those job descriptions.

In [1]:
df_jobs = pd.read_csv("../data/full_dataset/jul24_to_jul_2025_cleaned.csv")
df_jobs

NameError: name 'pd' is not defined

In [4]:
client = OpenAI(api_key=OPENAI_KEY)
tuples = list((skill, month, year) for skill, month, year in zip(df_jobs["Skills"], df_jobs["month"], df_jobs["year"]))

tuples.sort(key=lambda x: (x[2], x[1]))  
tuples = tuples[51001:]

time_to_skip = {(1, 2025), (3, 2025), (4, 2025)}

first_part_flag = True
job_skills_embeddings = []
save_every = 200
embedding_dim = 3072   

for i, tuple in enumerate(tqdm(tuples, desc="Generating embeddings for job skills", unit="embedding")):
    try:
        skill, month, year = tuple

        if (month, year) in time_to_skip:
            if first_part_flag:
                np.save("../data/embeddings/job_skills_embeddings_first_part.npy",
                        np.array(job_skills_embeddings, dtype=np.float32))
                job_skills_embeddings = []
            first_part_flag = False
            continue

        response = client.embeddings.create(
            input=skill,
            model="text-embedding-3-large"
        )
        emb = response.data[0].embedding

        if all(math.isfinite(x) for x in emb):
            job_skills_embeddings.append(emb)
        else:
            print(f"Invalid embedding on {i} — inserting zeros")
            job_skills_embeddings.append([0.0] * embedding_dim)

    except Exception as e:
        send_warning(f"Error generating embedding for job skill at index {i}: {e}")
        job_skills_embeddings.append([0.0] * embedding_dim)
        time.sleep(2)  

    if i > 0 and i % save_every == 0:
        if first_part_flag:
            np.save("../data/embeddings/job_skills_embeddings_first_part_partial_2.npy",
                    np.array(job_skills_embeddings, dtype=np.float32))
        else:
            np.save("../data/embeddings/job_skills_embeddings_second_part_partial_1.npy",
                    np.array(job_skills_embeddings, dtype=np.float32))
        print(f"Saved {len(job_skills_embeddings)} embeddings.")

np.save(f"../data/embeddings/job_skills_embeddings_second_part.npy",
        np.array(job_skills_embeddings, dtype=np.float32))

print("Done:", len(job_skills_embeddings), "embeddings saved.")


Generating embeddings for job skills:   0%|          | 201/153371 [00:53<14:08:40,  3.01embedding/s]

Saved 201 embeddings.


Generating embeddings for job skills:   0%|          | 401/153371 [01:47<11:04:59,  3.83embedding/s]

Saved 401 embeddings.


Generating embeddings for job skills:   0%|          | 601/153371 [02:36<10:04:09,  4.21embedding/s]

...

Saved 36043 embeddings.


Generating embeddings for job skills: 100%|█████████▉| 153201/153371 [6:55:45<03:33,  1.26s/embedding]

Saved 36243 embeddings.


Generating embeddings for job skills: 100%|██████████| 153371/153371 [6:56:38<00:00,  6.14embedding/s]


Done: 36413 embeddings saved.


## Join the three datasets
Given that we divided the dataset in three parts: 
* july 2024 to december 2024 (newly embedded)
* january 2025 to april 2025 (previously embedded for validation)
* may 2025 to july 2025 (newly embedded)
We need to join them into a single dataset for further processing.

In [ ]:
npy_array_1 = np.load("../data/embeddings/job_skills_embeddings_first_part_fixed_51002.npy")
npy_array_2 = np.load("../data/embeddings/job_skills_embeddings_first_part.npy")
npy_array_3 = np.load("../data/embeddings/old_embeddings/job_skills_embeddings.npy")
npy_array_4 = np.load("../data/embeddings/job_skills_embeddings_second_part.npy")

print("Shapes:", npy_array_1.shape, npy_array_2.shape, npy_array_3.shape, npy_array_4.shape)
total_embeddings = sum([npy_array_1.shape[0], npy_array_2.shape[0], npy_array_3.shape[0], npy_array_4.shape[0]])
print("Total embeddings:", total_embeddings)
print("Assertion:", total_embeddings + 1 == len(df_jobs)) # We eliminated one row in the jan-apr 2025 period because it was corrupted
print(len(df_jobs))

final_embeddings = np.concatenate([npy_array_1, npy_array_2, npy_array_3, npy_array_4], axis=0)
print("Final shape:", final_embeddings.shape)
np.save("../data/embeddings/full_job_skills_embeddings.npy", final_embeddings)

Shapes: (51002, 3072) (46639, 3072) (70318, 3072) (36413, 3072)
Total embeddings: 204372
Assertion: True
204373
Final shape: (204372, 3072)


In [37]:
full_dataset = pd.read_csv("../data/full_dataset/jul24_to_jul_2025_cleaned.csv")
full_dataset = full_dataset[~(full_dataset["month"].isin([1, 3, 4]) & (full_dataset["year"] == 2025))]
print("Final dataset shape:", full_dataset.shape)

Final dataset shape: (134054, 6)


In [39]:
df_jan_apr_2025 = pd.read_csv("../data/validation_green_skill_dataset/full_jan_apr_2025_dataset/jan_to_apr_2025_with_languages_cleaned.csv")
df_jan_apr_2025["year"] = 2025
print("Jan-Apr 2025 dataset shape:", df_jan_apr_2025.shape)

Jan-Apr 2025 dataset shape: (70318, 7)


In [40]:
full_dataset = pd.concat([full_dataset, df_jan_apr_2025], ignore_index=True)

In [41]:
print("Combined dataset shape:", full_dataset.shape)
print("Final assertion:", full_dataset.shape[0] == final_embeddings.shape[0])

Combined dataset shape: (204372, 7)
Final assertion: True


In [42]:
full_dataset = full_dataset.sort_values(by=["year", "month"]).reset_index(drop=True)
full_dataset.drop(columns=["detected_language"], inplace=True)
full_dataset.to_csv("../data/full_dataset/jul24_to_jul_2025_cleaned_sorted.csv", index=False)

Create the **FAISS** index and add the embeddings to it. Also save the index to disk.

In [43]:
# Create the embedding index

job_skills_embeddings = np.load("../data/embeddings/full_job_skills_embeddings.npy")

index_job_skills = faiss.IndexFlatIP(job_skills_embeddings[0].__len__())
job_skills_embeddings = np.array(job_skills_embeddings).astype('float32')
index_job_skills.add(job_skills_embeddings)

faiss.write_index(index_job_skills, "../data/embeddings/full_job_skills_embeddings.index")

In [44]:
print(index_job_skills.ntotal)

204372


Create a mapping between the index and the skill, similar to the one created for the green skills, here we are using tuples of the form `(i, (job_id, job_skill))`, where:
* `i` is the unique key for that entry, useful because **FAISS** only stores the *vector embedding*, not the actual text, so we need to save the identifier to map the vector to the entry.
* `(job_id, job_skill)`: a tuple with two elements, `job_id` is the unique identifier of the job posting, `job_skill` is the skill extracted from the job posting.

In [6]:
# Index for Job Skills
id_to_job = {i : (id, skill, month, year) for i, (id, skill, month, year) in enumerate(zip(full_dataset["Job_ID"], full_dataset["Skills"], full_dataset["month"], full_dataset["year"]))}
for i in range(5):
    print(id_to_job[i])

('job_624fd0a8f34770fb', 'experiencia en atencion al cliente', 7.0, 2024)
('job_624fd0a8f34770fb', 'gusto por el servicio y atencion al cliente', 7.0, 2024)
('job_624fd0a8f34770fb', 'preparatoria terminada', 7.0, 2024)
('job_624fd0a8f34770fb', 'mayor de anos', 7.0, 2024)
('job_624fd0a8f34770fb', 'experiencia en empleos presenciales', 7.0, 2024)


Save the mapping to a json file.

In [7]:
with open("../data/mapping/id_to_jobs.json", "w") as f:
    json.dump(id_to_job, f, indent=4)